In [1]:
import pandas as pd
from collections import defaultdict
from collections import Counter
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedShuffleSplit
#from sklearn.model_selection import train_test_split
#from sklearn.model_selection import StratifiedKFold

In [2]:
%matplotlib inline  
%autosave 300
pd.options.display.max_columns = None
feature_names =defaultdict(lambda:'None')

Autosaving every 300 seconds


# Classification based on published Data:

Risk High/low set at 38.40 Readiness high/low set at 46.51


In [3]:
def categorize(row):
    if row['overall_risk']== 1 and row['overall_readiness']==1:
        return 4
    elif row['overall_risk']== 1 and row['overall_readiness']==0:
        return 3
    elif row['overall_readiness'] == 1:
        return 2
    else:
        return 1
    
risk_ready=pd.read_csv('data/Overall Risk & Readiness Scores.csv')

risk_ready_old=risk_ready.columns
new_names=['city', 'state', 'geo_id','overall_risk', 'overall_readiness']
risk_ready.columns = new_names
    
# Puerto Rico is being dropped at rows [19, 35, 193, 220] because of data inconsistency
risk_ready.drop([19, 35, 193, 220], inplace = True)
risk_ready['overall_risk'] = np.where(risk_ready['overall_risk']>=.3840, 1, 0)
risk_ready['overall_readiness'] = np.where(risk_ready['overall_readiness']>=.4651, 1, 0)
risk_ready['risk_ready_cat']=risk_ready.apply(lambda row: categorize(row), axis=1)



risk_ready_dict = dict(zip(new_names, risk_ready_old))
feature_names['risk_ready']=risk_ready_dict



Clean main data File

In [4]:
city_ind = pd.read_csv('data/City Indicators.csv')     


city_ind_old=city_ind.columns
new_names=['city','state','geo_id','GDP_water_all','perc_fff','d_plan','w_plan','base_w_stress','invest','tax_break','corruption','perc_vote','innovate','climate_real','hs_ed','pop_dens','w_quality','debt','flood_pop','flood_build','flood_car','impervious','beds_1000','alone_65','child_5','health_ins','older_1999','mobile_home','rent_50plus','trees','heating','disabled','poverty','older_1979','work_outside','no_car','pop_under_1ft','pop_under_3ft','impact_plus1ft','impact_plus3ft', 'col_empty','lat','long','county','region','city_km2','pop_2010','pop_2011','pop_2012','pop_2013','pop_2014','pop_2015','median_income']
city_ind.columns = new_names

city_ind.drop(columns =['col_empty'], inplace= True)
indicators_dict = dict(zip(city_ind.columns, city_ind_old))



#Puerto Rico is being dropped at rows [19, 35, 193, 220] because of data inconsistency
city_ind.drop([19, 35, 193, 220], inplace = True)
#two cities had their order of magnitude incorrect for % of people in floodzone. Their neighboring values match the lower magnitude'
city_ind=city_ind.replace(412.9208681,41.29208681)
city_ind=city_ind.replace(101.6959823,10.16959823)
city_ind.replace('Bayam<U+0086>n','Bayamon')
city_ind['risk_ready_cat']= risk_ready['risk_ready_cat']


city_ind.drop(columns=['city','state','geo_id','county', 'region','pop_2010', 'pop_2011', 'pop_2012', 'pop_2013', 'pop_2014'], inplace=True)
city_ind['d_plan'] = city_ind['d_plan'].map({'Yes': 1, 'No': 0})
city_ind['w_plan'] = city_ind['w_plan'].map({'Yes': 1, 'No': 0})
city_ind.fillna(0, inplace=True)

city_ind.groupby('risk_ready_cat').count()


,GDP_water_all,perc_fff,d_plan,w_plan,base_w_stress,invest,tax_break,corruption,perc_vote,innovate,climate_real,hs_ed,pop_dens,w_quality,debt,flood_pop,flood_build,flood_car,impervious,beds_1000,alone_65,child_5,health_ins,older_1999,mobile_home,rent_50plus,trees,heating,disabled,poverty,older_1979,work_outside,no_car,pop_under_1ft,pop_under_3ft,impact_plus1ft,impact_plus3ft,lat,long,city_km2,pop_2015,median_income
risk_ready_cat,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48
2,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80
3,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87,87
4,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59


In [5]:

trees_cat=city_ind['risk_ready_cat']
trees_data = city_ind.drop(columns=['risk_ready_cat'])
tree_features=trees_data.columns


    prob=rf.predict_proba(X_test)
    fpr,tpr,thresholds = roc_curve(y_test, prob[:, 1])
    tprs.append(interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw=1, alpha=0.3,
             label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))
    i += 1

    plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
         label='Chance', alpha=.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
plt.plot(mean_fpr, mean_tpr, color='b',
         label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
         lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                 label=r'$\pm$ 1 std. dev.')

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()


In [9]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.multiclass import OneVsRestClassifier
from sklearn import metrics

import numpy as np
from scipy import interp
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
X=np.array(trees_data)
y=np.array(trees_cat)
sss = StratifiedShuffleSplit(n_splits=5, random_state=42)
sss.get_n_splits(X,y)

     
rf_scores =[]
tprs= []
aucs=[]
'''
RandomForestClassifier(n_estimators=’warn’, criterion=’gini’, max_depth=None, min_samples_split=2, 
min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=’auto’, max_leaf_nodes=None,
min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, 
n_jobs=None, random_state=None, verbose=0, warm_start=False, class_weight=None)
'''


for train_index, test_index in sss.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    rf = OneVsRestClassifier(RandomForestClassifier(n_estimators=100, random_state=42))
    rf.fit(X_train, y_train)
    y_hat=rf.predict(X_test)
    score=rf.score(X_test,y_test)
    rf_scores.append(score)
    print(score)
    
rf_mean_accuracy=sum(rf_scores)/len(rf_scores)
print(f'rf mean accuracy= {rf_mean_accuracy}')
rf_feat=pd.DataFrame()

rf_feat['features']=trees_data.columns


    


0.6071428571428571
0.7857142857142857
0.7857142857142857
0.6428571428571429
0.7142857142857143
rf mean accuracy= 0.7071428571428571


In [12]:
from sklearn.ensemble import AdaBoostClassifier
'''
AdaBoostClassifier(base_estimator=None, n_estimators=50, learning_rate=1.0, algorithm=’SAMME.R’, 
random_state=None
'''
ada_scores =[]
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    ada = OneVsRestClassifier(AdaBoostClassifier(random_state=42))
    ada.fit(X_train, y_train)
    y_hat=ada.predict(X_test)
    score=ada.score(X_test,y_test)
    ada_scores.append(score)
    print(score)
    
ada_mean_accuracy=sum(ada_scores)/len(ada_scores)

print(f'ada boost mean accuracy= {ada_mean_accuracy}')
ada_feat=pd.DataFrame()

ada_feat['features']=trees_data.columns
#ada_feat['Importance']=rf.feature_importances_


0.6428571428571429
0.6785714285714286
0.6785714285714286
0.75
0.75
ada boost mean accuracy= 0.7


In [ ]:


data = np.random.rand(5, 10)  # 5 entities, each contains 10 features
label = np.random.randint(2, size=5)  # binary target
dtrain = xgb.DMatrix(data, label=label)